In [32]:
from constructor.db import Collection
from constructor.table import Table
import pandas as pd
import numpy as np # linear algebra
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

import os
import warnings
warnings.filterwarnings(action='once')

## To simulate big data and RAM problems, additional libraries are used.
Making this mandatory part possible was achieved using Pandas and the PyTables library. The code below initiates a Table class which contains the following functions: 
```python
class Table:
    def __init__(self, csv_file, hdf_file, hdf_key, df_cols_to_index=True):
        self.csv_file = csv_file
        self.hdf_file = hdf_file
        self.hdf_key = hdf_key
        self.df_cols_to_index = df_cols_to_index

    def store(self):
        # Delete hdf file if it exists already.
        if path.exists(self.hdf_file):
            try:
                remove(self.hdf_file)
            except OSError:
                pass
        store = pd.HDFStore(self.hdf_file)
        for chunk in pd.read_csv(self.csv_file, chunksize=500000):
            # don't index data columns in each iteration
            store.append(self.hdf_key, chunk, data_columns=self.df_cols_to_index, index=False)
        # index data columns in HDFStore
        store.create_table_index(self.hdf_key, columns=self.df_cols_to_index, optlevel=9, kind='full')
        store.close()
        return store
```

Using this function a HDF file is generated based on a CSV-input file which also spawns a DataFrame.

In [33]:
review_table = Table('hotel-reviews.csv', './data/table.h5', 'hotel_reviews')
review_table.store()

ValueError: The file './data/table.h5' is already opened, but in read-only mode.  Please close it before reopening in append mode.

### Sentiment int, additional field indicating a positive Review 1, or a negative review 0
To add the sentiment column to the dataframe, it's mandatory to split the positve and negative reviews first. Therefore the reviews are split based on the Reviewer's score. Above 5.4 is positive and below 5.5 is negative. These df's will be concatinated back later on.

In [3]:
positive = review_table.query('Reviewer_Score > 5.4')
positive['Sentiment'] = 1
negative = review_table.query('Reviewer_Score < 5.5')
negative['Sentiment'] = 0

/Users/joost/collector/hva-data-scientist/lib/python3.7/site-packages/tables/leaf.py:414: PerformanceWarning: The Leaf ``/hotel_reviews/_i_table/Negative_Review/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)
/Users/joost/collector/hva-data-scientist/lib/python3.7/site-packages/tables/leaf.py:414: PerformanceWarning: The Leaf ``/hotel_reviews/_i_table/Positive_Review/sorted`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (

### To achieve the requested structure, both the Positive and Negative reviews should be available under Review_Text. Therefore the columns are renamed and the opposite columns will be dropped.


In [4]:
positive.rename(columns={'Positive_Review': 'Review_Text'}, inplace=True)
positive.drop("Negative_Review", axis=1)
negative.rename(columns={'Negative_Review': 'Review_Text'}, inplace=True)
negative.drop("Positive_Review", axis=1)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Review_Text,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Sentiment
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,0
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,0
6,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/17/2017,7.7,Hotel Arena,United Kingdom,Cleaner did not change our sheet and duvet ev...,33,1403,18,6,4.6,"[' Leisure trip ', ' Group ', ' Duplex Twin Ro...",17 days,52.360576,4.915968,0
12,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/6/2017,7.7,Hotel Arena,United Kingdom,The floor in my room was filfy dirty Very bas...,28,1403,6,7,4.6,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",28 days,52.360576,4.915968,0
21,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,6/12/2017,7.7,Hotel Arena,France,Our room was an overrated disaster room 231 d...,92,1403,75,12,3.8,"[' Couple ', ' Duplex Double Room ', ' Stayed ...",52 days,52.360576,4.915968,0
22,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,6/10/2017,7.7,Hotel Arena,Germany,Sadly I cannot say that the rooms are clean e...,364,1403,37,21,5.4,"[' Leisure trip ', ' Family with young childre...",54 days,52.360576,4.915968,0
32,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,5/11/2017,7.7,Hotel Arena,United States of America,Our bathroom had an urine order Shower was ve...,68,1403,0,1,4.2,"[' Leisure trip ', ' Group ', ' Duplex Twin Ro...",84 days,52.360576,4.915968,0
42,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,4/14/2017,7.7,Hotel Arena,Switzerland,Hotel is going through a major construction r...,110,1403,15,4,2.9,"[' Business trip ', ' Solo traveler ', ' Large...",111 day,52.360576,4.915968,0
44,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,4/5/2017,7.7,Hotel Arena,United States of America,The service was awful They refused to take ow...,26,1403,11,1,3.3,"[' Leisure trip ', ' Group ', ' Large King Roo...",120 day,52.360576,4.915968,0
46,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,4/5/2017,7.7,Hotel Arena,United Kingdom,The hotel is under construction which was nev...,315,1403,3,7,4.2,"[' Leisure trip ', ' Couple ', ' Duplex Double...",120 day,52.360576,4.915968,0


### After some initial selection cleaning the result should be written away as a Review_pos.csv and Review_neg.csv
Below code will transform the postive and negative df's to a CSV-file.

In [5]:
positive.to_csv(r'./data/Review_pos.csv', index=None, header=True)
negative.to_csv(r'./data/Review_neg.csv', index=None, header=True)

As told earlier, the dataframes need to be concatinated back together to from a complete df including the sentiment values.

In [6]:
df = pd.concat([positive, negative], sort=False)

### There should be a collection of balanced set of reviews, for instance a collection consisting of 10.000 positive and 10.000 negative reviews having a least the following structure
* Review
* Sentiment

Code below samples the positive and negative collections with 10K records each. These and the full collection will be filled into the MongoDB instance using the Collection class:
```python
class Collection:
    def __init__(self, df, collection):
        self.client = MongoClient('mongodb://localhost:27017')  # Init mongoclient
        self.db = self.client['assignment-2']  # Assign db name
        self.df = df
        self.collection = collection
        self.conn = self.db[self.collection]

    def fill(self):
        dict = self.df.to_dict(orient='records')
        self.conn.delete_many({ })    # Clear collection using an empty filter.
        self.conn.insert_many(dict)
```

In [7]:
Collection(df, 'hotel-reviews').fill()    # Fill mongo with complete df
Collection(positive[['Review_Text', 'Sentiment']].sample(n=10000), 'pos').fill()
Collection(negative[['Review_Text', 'Sentiment']].sample(n=10000), 'neg').fill()